# 目標
* 「html構造」→「trafilatura」→「できるだけ分解した処理群」→「MLベース前」
* trafilaturaの処理
* 処理群の分解

* Mecabのエラーについて：辞書のインストールが必要になるが、[この記事](https://analytics-note.xyz/programming/python-mecabrc-error/)と全く同じ現象が起きていた。

# 準備

In [4]:
!pip install --quiet (warcio)
!pip install --quiet (beautifulsoup4)
!pip install --quiet (trafilatura)
!pip install --quiet (mecab-python3)
!pip install --quiet (unidic)
!pip install --quiet (unidic-lite)

zsh:1: number expected


zsh:1: unknown file attribute: b
zsh:1: number expected
zsh:1: number expected
zsh:1: missing delimiter for 'u' glob qualifier
zsh:1: missing delimiter for 'u' glob qualifier


In [1]:
#Mecabの動作チェック　
#ここでエラーをはいた場合、形態素解析を伴う処理はスキップしてください。
import MeCab
mecab = MeCab.Tagger()
print(mecab.parse("私は自然言語処理分野の研究をしています。"))


私	代名詞,,,,,,ワタクシ,私-代名詞,私,ワタクシ,私,ワタクシ,和,"","","","","","",体,ワタクシ,ワタクシ,ワタクシ,ワタクシ,"0","","",11345327978324480,41274
は	助詞,係助詞,,,,,ハ,は,は,ワ,は,ワ,和,"","","","","","",係助,ハ,ハ,ハ,ハ,"","動詞%F2@0,名詞%F1,形容詞%F2@-1","",8059703733133824,29321
自然	名詞,普通名詞,一般,,,,シゼン,自然,自然,シゼン,自然,シゼン,漢,"","","","","","",体,シゼン,シゼン,シゼン,シゼン,"0","C2","",4227355954520576,15379
言語	名詞,普通名詞,一般,,,,ゲンゴ,言語,言語,ゲンゴ,言語,ゲンゴ,漢,"","","","","","",体,ゲンゴ,ゲンゴ,ゲンゴ,ゲンゴ,"1","C1","",3205085018595840,11660
処理	名詞,普通名詞,サ変可能,,,,ショリ,処理,処理,ショリ,処理,ショリ,漢,"","","","","","",体,ショリ,ショリ,ショリ,ショリ,"1","C3","",4691624739348992,17068
分野	名詞,普通名詞,一般,,,,ブンヤ,分野,分野,ブンヤ,分野,ブンヤ,漢,"","","","","","B",体,ブンヤ,ブンヤ,ブンヤ,ブンヤ,"1","C1","",9249925070201344,33651
の	助詞,格助詞,,,,,ノ,の,の,ノ,の,ノ,和,"","","","","","",格助,ノ,ノ,ノ,ノ,"","名詞%F1","",7968444268028416,28989
研究	名詞,普通名詞,サ変可能,,,,ケンキュウ,研究,研究,ケンキュー,研究,ケンキュー,漢,"","","","","","",体,ケンキュウ,ケンキュウ,ケンキュウ,ケンキュウ,"0","C2","",3104754582561280,11295
を	助詞,格助詞,,,,,ヲ,を,を,オ,を,オ,和,"","","","","","",格助,ヲ,ヲ,ヲ,ヲ,"","動詞%F2@0,名詞%F1,形容詞%F2@-1","",11381878116459008,4140

### WARC形式の読み込み

In [1]:
# WARC形式の読み込み
import pandas as pd
path="/data/fairy_hdd4/takumi/matsuo_llm/Dataset_for_BTM/01web_codes/processes/CC-MAIN-2023-50_text_batch0_CC-MAIN-2023-50-batch0-iter0.parquet"
df=pd.read_parquet(path)
print(df.head())
df=df[["content","url"]]

                                                 url  \
0             http://feathercloud.blks.jp/faq/04/01/   
1  http://fline.biz/list/detail_gourmet/post-108.php   
2     http://gagon.com/videos/xha/xha023/xhs023.html   
3      http://ikechang.com/news/2008/news0809j5.html   
4                               http://ks-com.co.jp/   

                                       title             timestamp  \
0                  Feathercloud [よくある質問-FAQ]  2023-11-28T10:19:01Z   
1  ホームライフサポート｜一関市　フリーペーパー【FREE LINE】地域情報が満載！  2023-11-28T11:18:14Z   
2        Fetish Factory Videos Hair Scene019  2023-11-28T10:21:42Z   
3           天童のニュース（平成20年9月・第5週） - 天童市の観光ガイド  2023-11-28T10:52:42Z   
4                              K'S COM GROUP  2023-11-28T10:07:26Z   

                                             content  
0  b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01...  
1  b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0...  
2  b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0...  
3  b'<!DOCTYPE HTML PU

In [5]:
text1=df['content'].iloc[1].decode("UTF-8")
text1 = extract(text1, include_tables=False,target_lang='ja',favour_precision=True) #trafilaturaでテキスト抽出

### 変換

In [7]:
#変換
texts=[]
euc_jp_list=[3,29,73]
utf_list=[0,1,2,5,8,10,11,12,14,16,17,19,21,22,24,27,30,31,32,33,34,36,37,38,
          39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,55,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,94,95,96,97,99,100,101,103]
shift_jis_list=[4,7,9,13,15,18,23,25,26,28,35,54,56,74,90,92,93,102] #6,20,98
for i in range(len(df)):
    # print(i)
    if i in euc_jp_list:
        text=df['content'].iloc[i].decode("EUC-JP")
        #print("EUC-JP")
        texts.append(text)
    elif i in shift_jis_list:
        text=df['content'].iloc[i].decode("Shift_JIS")
        #print("Shift_JIS")
        texts.append(text)
    elif i in [6,20,98]:
        #text=df['content'].iloc[i].decode("shift_jis")
        # print("no")
        pass
    else:# i in utf_list:
        text=df['content'].iloc[i].decode("UTF-8")
        # print("UTF-8")
        texts.append(text)
    


# clean_textパイプライン
- lang
- url
- trafilatura

- hoji_filter(分割すべき？)
    - コンテンツなど
- prob_hoji_filter
- normlize
- repetition1
- repetition2
- line-wise(処理を増やす)
- remove_multi_headers
- rule_based_text_checker

In [ ]:
#順番

# def clean_text(text):
#     text=extract_by_trafilatura(text)
#     text=hojifilter(text)
#     text=normalize(text)
#     text=repetition1(text)
#     text=repetition2(text)　#処理重い
#     text=line_wise(text)　#処理重い
#     text=remove_multi_headers(text)
#     text=rule_based_text_checker(text)

## 関数：extract_by_trafilatura

In [10]:
from trafilatura import fetch_url, extract
def extract_by_trafilatura(text):
    text = extract(text1, include_tables=False,target_lang='ja',favour_precision=True) #trafilaturaでテキスト抽出
    return text

## check

In [ ]:
# text checker
def char_is_hiragana(c):
    return u'\u3040' <= c <= u'\u309f'

def contains_hiragana(s):
    return any(char_is_hiragana(c) for c in s)

def check(s):
    if not contains_hiragana(s):
        return ""

    else:
        return s

## 関数：hojichar_filter

In [12]:
#hojichar
from hojichar import Compose, document_filters
import json
import random
import sys
sys.path.append("/data/fairy_hdd4/takumi/matsuo_llm/Dataset_for_BTM/01web_codes/src")

base_path = "/data/fairy_hdd4/takumi/matsuo_llm/Dataset_for_BTM/01web_codes/src/cleaner/hoji_dict/"
cleaner = Compose([
    document_filters.JSONLoader(key="text"),
    document_filters.AcceptJapanese(),
    document_filters.DocumentLengthFilter(min_doc_len=10, max_doc_len=50000),
    document_filters.MaskPersonalInformation(),
    document_filters.JSONDumper(),
])

with open(base_path + "adult_keywords_ja.txt") as f:
    adult_keywords_ja = f.read().splitlines()
with open(base_path + "adult_keywords_en.txt") as f:
    adult_keywords_en = f.read().splitlines()
with open(base_path + "advertisement_keywords_ja.txt") as f:
    advertisement_keywords_ja = f.read().splitlines()

noise_keywords = adult_keywords_ja + adult_keywords_en + advertisement_keywords_ja
noise_keywords = list(set(noise_keywords))
noise_keywords = [k for k in noise_keywords if k != ""]

prob_cleaner = Compose([
    document_filters.JSONLoader(key="text"),
    # document_filters.DiscardRareKuten(),  # 日本語以外を消す
    document_filters.DiscardAdultContentJa(
        base_path + "adult_keywords_ja.txt"),
    document_filters.DiscardAdultContentEn(
        base_path + "adult_keywords_en.txt"
    ),
    # document_filters.DiscardDiscriminationContentJa(
    #    base_path + "discrimination_keywords_ja.txt"
    # ),
    # document_filters.DiscardViolenceContentJa(
    #    base_path + "violence_keywords_ja.txt"
    # ),
    document_filters.DiscardBBSComments(),
    document_filters.DiscardAds(
        base_path + "advertisement_keywords_ja.txt",
        max_allowed_num=10,
    ),
    document_filters.JSONDumper(),
])

def hoji_filter(text):
    d = {"text": text}
    parsed = cleaner(json.dumps(d))
    if parsed == "":
        return ""
    text = json.loads(parsed)["text"]
    return text


# 確率的にフィルタリングするが､ちょっとイマイチ
def prob_hoji_filter(text, survive_ratio=0.5):
    # ngワード類については､確率的に処理する
    if random.random() < survive_ratio:
        return text

    d = {"text": text}
    parsed = prob_cleaner(json.dumps(d))
    if parsed == "":
        return ""
    text = json.loads(parsed)["text"]
    return text


def prob_filter(text, ratio=5):
    # 遅い
    if not text:
        return ""

    # テキスト全体の長さ
    total_length = len(text)

    # noise_keywordsの各単語がテキストに含まれる回数の合計を計算
    noise_length = sum(text.count(word) for word in noise_keywords)

    # noise_keywordsがテキストに占める割合を計算
    noise_ratio = noise_length / total_length

    # print(noise_ratio*ratio)
    # 割合が閾値以上であれば、確率的に""を返す
    if noise_ratio*ratio > random.random():
        return ""
    else:
        return text


## 関数：normalize

In [13]:
# https://github.com/lighttransport/japanese-llama-experiment/blob/main/02_normalize/text_normalizer.py

# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.
#
# Added some modification for Japanese text normalization.

import re
import unicodedata

#
# NOTE: 実際のところ, 01_normalize では PUNCT 変換は行わない.
# dedup 時には行う(PUNCT 除去)
# TODO: dedup 時, 日本語の場合は句読点を含んだままのほうが良いような気もするがどうか.
#

UNICODE_PUNCT = {
    # 日本語の場合は句読点は、。のままがよいでしょう
    "。": "。",
    "、": "、",
    "，": "、",

    "„": '"',
    "”": '"',
    "“": '"',
    "«": '"',
    "»": '"',
    "１": '"',
    "」": '"',
    "「": '"',
    "《": '"',
    "》": '"',
    "´": "'",
    "∶": ":",
    "：": ":",
    "？": "?",
    "！": "!",
    "（": "(",
    "）": ")",
    "；": ";",
    "–": "-",
    "—": " - ",
    "．": ". ",
    "～": "~",
    "’": "'",
    "…": "...",
    "━": "-",
    "〈": "<",
    "〉": ">",
    "【": "[",
    "】": "]",
    "％": "%",
    "►": "-",
}

UNICODE_PUNCT_RE = re.compile(f"[{''.join(UNICODE_PUNCT.keys())}]")


def replace_unicode_punct(text: str) -> str:
    return "".join((UNICODE_PUNCT.get(c, c) for c in text))


def remove_unicode_punct(text: str) -> str:
    """More aggressive version of replace_unicode_punct but also faster."""
    return UNICODE_PUNCT_RE.sub("", text)


# Reuse `strip_accents` for CJK text. Use NFKC
def strip_accents(line: str) -> str:
    """Strips accents from a piece of text."""
    # nfd = unicodedata.normalize("NFD", line)
    nkfc = unicodedata.normalize("NFKC", line)
    output = [c for c in nkfc if unicodedata.category(c) != "Mn"]
    if len(output) == line:
        return line
    return "".join(output)


# Build a regex matching all control characters.
# newline(LF, 10) has meaningful infor in CJK text, so do not remove it.
NON_PRINTING_CHARS_RE = re.compile(
    f"[{''.join(map(chr, list(range(0,10)) + list(range(11, 32)) + list(range(127,160))))}]"
)
DIGIT_RE = re.compile(r"\d")
PUNCT_OR_NON_PRINTING_CHARS_RE = re.compile(
    (UNICODE_PUNCT_RE.pattern +
     NON_PRINTING_CHARS_RE.pattern).replace("][", "")
)


def remove_non_printing_char(text: str) -> str:
    return NON_PRINTING_CHARS_RE.sub("", text)


def normalize_spacing_for_tok(text: str, language: str = "en") -> str:
    res = (
        text.replace("\r", "")
        # remove extra spaces
        .replace("(", " (")
        .replace(")", ") ")
        .replace(" +", " ")
    )
    res = re.sub(r"\) ([\.\!\:\?\;\,])", r"\)\1", res)
    res = res.replace("( ", "(").replace(" )", ")")
    res = re.sub(r"(\d) \%", r"\1\%", res)
    res = res.replace(" :", ":").replace(" ;", ";")
    res = res.replace("`", "'").replace("''", ' " ')

    res = (
        res.replace("„", '"')
        .replace("“", '"')
        .replace("”", '"')
        .replace("–", "-")
        .replace("—", " - ")
        .replace(" +", " ")
        .replace("´", "'")
        .replace("([a-z])‘([a-z])", r"\1'\2/")
        .replace("([a-z])’([a-z])", r"\1'\2/")
        .replace("‘", '"')
        .replace("‚", '"')
        .replace("’", '"')
        .replace("''", '"')
        .replace("´´", '"')
        .replace("…", "...")
        # French quotes
        .replace(" « ", ' "')
        .replace("« ", '"')
        .replace("«", '"')
        .replace(" » ", '" ')
        .replace(" »", '"')
        .replace("»", '"')
        # handle pseudo-spaces
        .replace(" %", "%")
        .replace("nº ", "nº ")
        .replace(" :", ":")
        .replace(" ºC", " ºC")
        .replace(" cm", " cm")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ;", ";")
        .replace(", ", ", ")
        .replace(" +", " ")
        .replace("．", ". ")
    )
    # English "quotation," followed by comma, style
    if language == "en":
        res = re.sub(r"\"([,\.]+)", r"\1\"", res)
    # Czech is confused
    elif language == "cs" or language == "cz":
        pass
    # German/Spanish/French "quotation", followed by comma, style
    else:
        res = res.replace(',"', '",')
        res = re.sub(
            r"(\.+)\"(\s*[^<])", r"\"\1\2", res
        )  # don't fix period at end of sentence

    if (
        language == "de"
        or language == "es"
        or language == "cz"
        or language == "cs"
        or language == "fr"
    ):
        res = re.sub(r"(\d) (\d)", r"\1,\2", res)
    else:
        res = re.sub(r"(\d) (\d)", r"\1.\2", res)
    return res


# NOTE accent=True will do NFKC normalization
# NOTE: set punct=0(no zenkaku->hankaku conversion) hby default for Japanese dataset
def normalize(line: str, accent=True, case=False, numbers=False, punct=0) -> str:
    line = line.strip()
    if not line:
        return line
    if case:
        line = line.lower()

    # FIXME: Always apply NKFC normalization for CJK text.
    if accent:
        line = strip_accents(line)
    if numbers:
        line = DIGIT_RE.sub("0", line)
    if punct == 1:
        line = replace_unicode_punct(line)
    elif punct == 2:
        line = remove_unicode_punct(line)
    line = remove_non_printing_char(line)
    return line


def slow_normalize_for_dedup(line: str) -> str:
    return normalize(line, accent=False, case=True, numbers=True, punct=2)


def normalize_for_dedup(line: str) -> str:
    line = line.strip()
    if not line:
        return line
    # case
    line = line.lower()
    # numbers
    line = DIGIT_RE.sub("0", line)
    line = PUNCT_OR_NON_PRINTING_CHARS_RE.sub("", line)
    return line

## 関数：repetition_filter1,2

In [61]:
from collections import Counter

# 単一文字の繰り返し200----------------------------------------


def repetition_filter1(text, threshold_ratio=0.3):
    # # 文字の繰り返しをチェックする辞書
    char_count = {}
    # 各文字の繰り返し回数をカウント
    for char in text:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1
    # ratio%以上繰り返される文字があるかチェック
    #print(text)
    repeated_chars = [char for char, count in char_count.items() if count >= threshold_ratio*len(text)]
    # 繰り返される文字がなければmatrix_tempに追加
    if not repeated_chars:
        return text
    return ""


def repetition_filter2(text, thresholds={
    'line_dup': 0.30,
    'paragraph_dup': 0.30,
    'char_in_line_dup': 0.20,
    'char_in_paragraph_dup': 0.20,
    '2-gram': 0.20,
    '3-gram': 0.18,
    '4-gram': 0.16,
    '5-gram': 0.15,
    '6-gram': 0.14,
    '7-gram': 0.13,
    '8-gram': 0.12,
    '9-gram': 0.11,
        '10-gram': 0.10}):
    if text == "":
        return ""
    # 段落と行に分割
    paragraphs = text.split('\n')
    lines = text.replace('\n', ' ').split('。')

    # 段落と行の重複率を計算
    paragraph_dup_rate = calc_dup_rate(paragraphs)
    line_dup_rate = calc_dup_rate(lines)

    # 文字に含まれる重複の割合を計算
    char_in_paragraph_dup_rate = calc_char_dup_rate(paragraphs, text)
    char_in_line_dup_rate = calc_char_dup_rate(lines, text)

    # n-gramの重複率を計算
    ngram_dup_rates = {}
    for n in range(2, 11):
        ngrams = extract_ngrams(text.replace('\n', ''), n)
        if n < 5:
            # 最頻出のn-gramの出現回数を計算
            ngram_dup_rates[n] = calc_max_freq_rate(ngrams)
        else:
            # 2回以上出現するn-gramの総出現回数を計算
            ngram_dup_rates[n] = calc_total_dup_freq_rate(ngrams)

    # 各指標が閾値を超えているかチェック
    if (line_dup_rate > thresholds['line_dup'] or
        paragraph_dup_rate > thresholds['paragraph_dup'] or
        char_in_line_dup_rate > thresholds['char_in_line_dup'] or
        char_in_paragraph_dup_rate > thresholds['char_in_paragraph_dup'] or
            any(ngram_dup_rates[n] > thresholds[f'{n}-gram'] for n in range(2, 11))):
        # print(text)
        return ""

    return text


def calc_dup_rate(items):
    counter = Counter(items)
    total = len(items)
    dup_count = sum(1 for count in counter.values() if count > 1)
    return dup_count / total if total > 0 else 0


def calc_char_dup_rate(items, text):
    counter = Counter(items)
    total_chars = len(text)
    dup_chars = sum(len(item) * (count - 1)
                    for item, count in counter.items() if count > 1)
    return dup_chars / total_chars if total_chars > 0 else 0


def extract_ngrams(text, n):
    return [text[i:i+n] for i in range(len(text)-n+1)]


def calc_max_freq_rate(ngrams):
    counter = Counter(ngrams)
    total = len(ngrams)
    max_freq = max(counter.values())
    return max_freq / total if total > 0 else 0


def calc_total_dup_freq_rate(ngrams):
    counter = Counter(ngrams)
    total = len(ngrams)
    dup_freq = sum(count - 1 for count in counter.values() if count > 1)
    return dup_freq / total if total > 0 else 0


## 関数：line_wise

### splitter

In [31]:
from typing import List
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation


def text_to_paragraph_sentences(text: str) -> List[List[str]]:
    """
    Split a text into paragraphs and sentences.

    Args:
        text (str): The text to split.

    Returns:
        List[List[str]]: The paragraphs and sentences.
    """
    paragraphs = paragraph_split(text)
    return [sentence_split(paragraph) for paragraph in paragraphs]


def paragraph_split(text: str) -> List[str]:
    """
    Split a text into paragraphs.

    Args:
        text (str): The text to split.

    Returns:
        List[str]: The paragraphs.
    """
    return text.split("\n")


split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_te = functools.partial(
    concatenate_matching, remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline,
                          concat_tail_te, split_punc2)


def sentence_split(text: str) -> List[str]:
    """
    Split a text into sentences.

    Args:
        text (str): The text to split.

    Returns:
        List[str]: The sentences.
    """
    return list(segmenter(text))


In [15]:
# import sys
# sys.path.append("/data/fairy_hdd4/takumi/matsuo_llm/Dataset_for_BTM/01web_codes/src/cleaner")
# from .splitter import text_to_paragraph_sentences
# # from .text_normalizer import normalize
# #from . import text_checker
# from . import rule_based_line_checker
# # from . import parts_filter
# from .line_end_cleaner import clean_line_endings
# #from .hojichar_filter import hoji_filter, prob_hoji_filter, prob_filter
# #from . import rule_based_text_checker
# from .line_dedup import remove_multi_headers
# #from . import repeated_phrase
# # try:
#     from .TextClassifier import TextClassifier
#     classifier = TextClassifier()
# except:
#     print("error loading TextClassifier. install fasttext to use it")

def text_to_cleaned_paragraphs(text):
    text = normalize(text)  # 正規化
    # text = text_checker.check(text)  # ひらがなを含まないテキストは除外

    # 繰り返し表現を除外 by Naito
    text = repeated_phrase.repeated_id(text)
    text = repeated_phrase.is_repetitive_japanese(
        text)  # n-gramの計算(計算量が多そうな場合、削る)

    # パラグラフと文章に分割
    paragraphs = text_to_paragraph_sentences(text)

    new_paragraphs = []
    for paragraph in paragraphs:
        new_lines = []
        old_line = ""
        for line in (paragraph):
            # ルールベースの行チェック
            new_line = rule_based_line_checker.clean(line)

            # 名詞だらけのlineを除外
            try:
                # new_line = parts_filter.filter(new_line)
                new_line = parts_filter.filter2(new_line)  # n-gramによる重複の除外
            except:
                pass
            if new_line:
                if new_line == old_line:
                    continue
                old_line = new_line
                # ppl=perp_checker(new_line)
                # print(ppl,new_line)
                new_lines.append(new_line)

        if new_lines:
            new_paragraphs.append(new_lines)

    # 文末が｡などでおわらないパラグラフ中の文章を削除
    # clean_line_endings(new_paragraphs)

    # パラグラフにまとめる
    cleaned_paragraphs = []
    old_lines = ""
    for paragraph in new_paragraphs:
        lines = "".join(paragraph)
        if lines == old_lines:
            continue
        cleaned_paragraphs.append(lines)
        old_lines = lines

    return cleaned_paragraphs


def clean_text(text, hoji=True):
    if hoji:
        # text = prob_filter(text)
        text = hoji_filter(text)
        text = prob_hoji_filter(text)

    paragraphs = text_to_cleaned_paragraphs(text)
    # print("aa", original_text)
    text = "\n".join(paragraphs)

    text = remove_multi_headers(text)
    text = rule_based_text_checker.clean(text)
    text = text.strip()
    return text


def ml_clean_text(text):
    text = prob_hoji_filter(text)
    text = hoji_filter(text)
    text = classifier.clean(text)
    if text != "":
        text = clean_text(text, hoji=False)
    return text


### noise_line(rule-base-line-wise)

In [16]:
# https://github.com/lighttransport/japanese-llama-experiment/blob/main/03_clean_step1/clean_text.py

# TODO: hard codeしない
import random
import unicodedata
import re
broken_sentence_endings = """
...
... 
...　
\"
'
[…]
詳細を見る
すべて表示する
するまとめ
公式サイト
にアクセス!
お​問​い​合​わ​せ​
アクセスマップ
送料無料!
コメント(0)
トラックバックは利用できません。
この記事へのコメントはありません。
返信をキャンセルする。
URL
お問い合わせ
,?
周辺施設
続きを表示
音楽出版社求人
Check
マップMAP
未入力項目があります
SNS
●facebook→
●twitter→
ABOUT US
求人情報
1件中1～1件を表示
事業内容
トップページ
特集
賃貸情報
条件を選ぶ
現在の選択エリア
沿線から選ぶ
エリアを選びなおす
さらにエリアを絞る
△ページTOP
公式
About
search
プロフィール
更新情報をチェックする
検索:
関連記事
ブックマークする
友達に教える
お問い合せ
CLOSE
My account
Contact Us
ご利用方法
会員規約
特定商取引法に基づく表記
組織概要
<前の20件1次の20件>
官公庁臨時職員
湘南国際村仕事
タイアップ広告掲載
広告を掲載しませんか?
会社概要
Pocket
目次
TOP
から見つける
プライバシーポリシー
利用規約
All Rights Reserved.
滞在後投稿されたものです。
法令順守と犯罪抑止のために
メルカリあんしん・あんぜん宣言!
偽ブランド品撲滅への取り組み
スタッフブログ
賃貸ホームページ
物件カタログ
ホーム
メニュー
仲介手数料最大無料
口コミをもっと
詳細情報
ツイート
メルマガ
登録商標です。
ようこそゲストさん
→
[...]
の先頭へ
個人情報の取り扱いについて
中古マンション
»
パスワードをお忘れの方
上部へ
JavaScriptを有効にしてご利用下さい.
ログインできない時は?
運営会社
企業情報
問い合わせ
一覧へ
ヘルプ
物件
もっと見る
を探す
マンション
検索
店
検索のヒント:
新規の表示
Vths
Id
見出し語
Yomi
大分類1
小分類2
Pos
Actions
メールアドレス
パスワード
次回から自動ログインをする
パスワードを忘れた方はこちら
不動産無料査定
住まい紹介ブログ
🗣naamal
⋆。* ⋆。* ⋆
122投稿
サイトは?
トップ
ニュース
日程
著作権
記事・写真・動画の利用申込
採用情報
サイト一覧
関連情報
の投稿
ページトップへ
記事を読む
運営者情報
Twitter公式
もっとみる
投稿する
店舗TOP
スタッフ・開発者募集
ヘルプページ
最近のコメント
カレンダー
の情報:
の情報
いいね
キャンペーン
|BOOKED
HOME
よくある質問
BLOG
...
問い合わせください。
休診日
木曜日・日曜日・祝日
all rights reserved.
</tr>
</div>
浮気調査
カテゴリー
お知らせ
新着アイテム
アーカイブ
月
Search
TOPへ
TOPページへ
Menu
Return Top
携帯版はこちら
会社情報
初めての方へ
ダウンロード
アクセス
IR情報
一覧
採用サイト
マイページ
研究会会員
専用ページ
はこちら
相談する
相談
探す
事例を見る
お客様からの声(成功事例)
書籍
メタ情報
の詳細
Home
ログイン
投稿フィード
コメントフィード
船井総研のサービス
お客様からの声(成功事例)
レポート
サービス
船井総研のサービス概要
セミナー
経営研究会
コンサルティング
業種・テーマ
住宅・不動産
リフォーム
不動産
賃貸
建設
病院・クリニック
歯科医院
特定商取引法
はじめてお越しの方へ
LINE
menu
免責事項
閉じる
読者になる
リンク
Tweet
方法はコチラ
トップへ戻る
[ヘルプ]
のプロフィール
ランキング】
更新しました
アーカイヴ
未分類
お問合せ
シェア
[お問い合わせ]
[利用規約]
[個人情報保護方針]
/RSS
もっと読む
の検索結果
通販ショップ
フォローする
シェアする
MENU
このブログについて
コメントを書く
twitter
facebook
line
コメント
arrowup
読まれている記事
のビュー
質問箱
コメントをキャンセル
down
Website
more
Name
Email
home
新着情報
First
のQ&A
サイトへ
Previous
上へ
更新
Twitter
拡大
ランキング
Next
会員登録
Last
スタッフ紹介
1階層ページ
タイトル
この記事を削除する
2階層ページ
サイド見出し
送信する
イメージナビ
お客様の声・対策事例
ページ次のページ »
こちらから
役立つ文例集
お客様の声
個人情報保護ポリシー
特定商取引について
必須お名前
申込
フォーム
FAQ
電話する
ページ先頭へ
送信する
天気予報
ロフィールを表示
リンク集
上部へ戻る
へスキップ
Category
ご質問
ヤミ金の電話番号
最新記事
インフォメーション
コラム
事務所案内
サービス料金
リストナビ
掲示板BBS
地図ページ
メールフォーム
ケータイサイト
おすすめの注目情報を掲載。
この記事をツイートする
自動でツイートされます。
コメントをするには、
ログインして下さい。
ページの先頭へ戻る。
page
の口コミ
メニューにもどる。
調べる▼
検索結果
とURLをコピーしました
のご利用について
通販サイト
無料
無料ブログはココログ!
パスワードを忘れた方
特定商取引法表示
note(blog)
niconicoへのご意見・ご要望
ダウンロードはこちらからどうぞ。
下記のお問い合わせフォームでご連絡ください。
買い物かごカートに商品がありません。
ニュースレターに登録して最新情報やセール情報をいち早くゲット。
受信する受信しない
JavaScriptを有効にしてご覧ください。
このページの機能を利用するにはJavaScriptに対応したブラウザが必要です。
分で読めます。
無断転載を禁じます。
検索条件を入力してください。
コメント(0)トラックバックは利用できません。
この記事へのコメントはありません。返信をキャンセルする。
法令順守と犯罪抑止のためにメルカリあんしん・あんぜん宣言!Facebook
Facebook
はてなブログを使っています。
見つかりました
インプラント
歯科
VIPがお送りします
見積
ID:
PR
一覧
出会い
探せます
商品はございません
《前のページ|次のページ》
RSS
人気記事まだデータがありません。
投稿が見つかりませんでした。
投稿がありません。
別のサイトにジャンプしようとしています。
お問合せは24時間受け付けております。
お気軽にご連絡ください。
左の一覧からカテゴリーをお選びください。
までを表示します。

の人気クチコミ
JavaScriptをONにしてください。
サンプルです
リンクに追加する
サイドバー
これはテストです
のクチコミ
サイトについて
口コミ・感想!
ホーム>
Next Entry
"""

broken_ending_list = broken_sentence_endings.split("\n")
broken_ending_list = [x for x in broken_ending_list if len(x) > 0]

broken_ending_list += [
    '続きを読む', '[続きを読む]', '(続きを読む)', '続きを見る', '続きをみる', '(続く)', '(続きを表示)', '(続きをみる)', '[続きをみる]', '[続きを見る]',
]
broken_ending_list += ['...(続きを表示)', '[ 続きを見る ]',
                       '・・・続きを見る', '... 続きを読む',
                       "詳細はこちら »",
                       "詳細>>>",
                       "サイトマップ",
                       ]


noise_mid_list = """
Copyright(C)
デジタル広告ガイド
Copyright
HOME>
新聞広告ガイド
©
無断転載を禁止します
著作権
記事・写真・動画の利用申込
採用情報
Copyright © 
Copyright 
"""
noise_mid_list = noise_mid_list.split("\n")
noise_mid_list = [x for x in noise_mid_list if len(x) > 0]


random_mid_list = """
JavaScriptを
ページの先頭
登録・解除
お問い合わせ等
無料メールマガジン
配信登録
性癖
Blog
ブログ
ここに説明文が入ります
資料請求
メールですすめる
商品の状態
未使用
エリア・駅がありません
見つかりませんでした。
無料登録
人気記事
Top
新規登録
検索結果
当院
ホームページ
代表挨拶
チャンネル登録
借金
不動産
投資
カードローン
トップページ
Podcast
検索
賃貸
物件
Views
Clicks
.com
URL
更新日
配信
ランキング
徒歩
Javascriptの
記事一覧
ブログ
公開中
カート
このコンテンツ
パスワードで保護
サイト
公開物件
ホーム
予約
商品の
お探しの
サポートします
を掲載!
はこちら
レス数が1000を超えています
現在表示しているスレッド
このスレは
新着レスの表示
掲示板に戻る
前100
次100
最新50
Twitter
Facebook
はてブ
Pocket
LINE
ゲストさん
コメント(
コメント（
が提供
限定公開記事
本文へ
再度アクセス
メールでの
買い物を続ける
査定
個人情報保護
お買い上げ
印刷する
会員限定
プライバシー保護
無料
ご案内
mail
データ取得に失敗しました
@
このサイト
求人
タイムライン
SHARE
日記
閲覧
trackback
(-)
|
スポンサー広告
--------
page
top
Home
»
Loading.
URL
[編集]
▲
《
.jp
Escキー
楽天
営業
Yahoo!
メールアドレス
Share
フォロー
フォロワー
Youtube
VIEWS
購読する
最新情報
公開日
ちゃんねる
コンテンツ
購入手続
Twitter
@5ch
広告なし
周辺施設情報
賃貸マンション
Cookie
QRコード
出張
承ります
お任せください
タグ
ログイン
ユーザー登録
トップへ
食べログ
ご相談ください
全国対応
記事
お会計
リンク
ツール
引用
ストック
掲載
1 2 3 4 5
クリック
ご注文
メッセージ
お送りください
買い物
カート
RSS
全リスト
新着
いいね!
読む
とは?
CONTACT
エラーが発生しました
モバイル
お気に入り
もぐもぐ
利用者情報
運営者について
クリックで拡大
アイコン上
マウスカーソル
詳細情報
TikTok
特定商取引法
に基づく表示
プレイ日記
Q&A
よくあるご質問と回答
トピックス
背景色
メルマガ
(c)
javascript
dobe Reader
カテゴリ
ユーザー認証
早速見る
投稿
ナビゲーション
印刷
SNSでシェアしよう!
English
お問い合わせ
発送
業務
休日
状態
スペースキー
矢印キー
選択します
お客様各位
キャンペーン
お気に入り
探し方を変更する
掲示板
買い物を続ける
選んでください
配達日
選択してください
お電話
コメント
非公開
利用特定商取引法
基づく表示規約
お役立ち
個人情報の取扱
同意
Contact
URL
http
shares
ニュースレター
結果を見る
免責
PICK UP!
最近の記事
New!
Tweets
＝＝t?
をフォロー
お支払い
お選びいただけます。
ツイートする
すべて表示
お問合わせ
お客様
入会案内
事業全般
メールフォーム
カテゴリー
チェック
商品
お使いのブラウザ
JavaScript
店舗
登録
一覧
戻る
サービス
紹介
ページ
管理
送料
ショッピング
クレジットカード
ご利用
ガイド
お伝えします
ご紹介
お部屋探し
限定公開記事
再度お試し
利用規約
プライバシー
同意します
整骨院
クリニック
方針
Instagram
スマートフォン
ポリシー
contact
profile
void
会社案内
詳しく見る
ソーシャルメディア
1件
病院への地図
無料相談
文字を入力
過去の記事
メニューを閉じる
▶
Social
サイト
物件
メルカリ
ポイント
買取
ツイート
"""
random_mid_list = random_mid_list.split("\n")
random_mid_list = [x for x in random_mid_list if len(x) > 0]
random_mid_list = list(set(random_mid_list))


def noise_line(sent: str):
    for broken_ending in broken_ending_list:
        # print("aa", broken_ending)
        if sent.endswith(broken_ending):
            return None
        if len(sent) < 2:
            return None

    for noise_mid in noise_mid_list:
        if noise_mid in sent:
            return None

    # インターネットに特有の単語を含む文章を確率的に落とす
    for random_mid in random_mid_list:
        if random_mid in sent:
            if random.random() < 0.9:
                return None

    return sent

### parts filter(in Line wise)

In [18]:
import MeCab
from collections import Counter
wakati = MeCab.Tagger("-Owakati")
# テキスト
tagger = MeCab.Tagger()

def parts_count(text, return_word_count=False):
    # テキストを解析
    parsed = tagger.parse(text)

    # 品詞をカウントするためのCounterオブジェクト
    pos_counter = Counter()
    word_counter = Counter()

    # 解析結果を行ごとに処理
    all_counts = 0
    for line in parsed.split('\n'):
        # EOSまたは空行の場合はスキップ
        if line == 'EOS' or line == '':
            continue
        # タブで分割し、形態素情報を取得
        pos_info = line.split('\t')
        # print(pos_info)
        pos = pos_info[1]
        pos = pos.split(",")[0]

        if return_word_count:
            word = pos_info[0]
            word_counter[(word, pos)] += 1

        # 品詞をカウント
        pos_counter[pos] += 1
        all_counts += 1

    if return_word_count:
        return pos_counter, all_counts, word_counter
    else:
        return pos_counter, all_counts


def mecab_filter(text, threshold=0.9, min_length=10):
    """
    名詞の羅列の文章は無効と判定する
    """
    if text is None:
        return None
    if text == "":
        return None
    pos_counter, all_counts = parts_count(text)
    # print(pos_counter, all_counts)
    # print(pos_counter)
    meishi_and_symbol_counts = pos_counter['名詞'] + \
        pos_counter['記号']+pos_counter['補助記号']+pos_counter['接頭詞']

    if all_counts == 0:
        return None
    ratio = meishi_and_symbol_counts/all_counts
    # print(ratio, pos_counter)
    if ratio > threshold and len(text) > min_length:
        return None
    else:
        return text


def n_gram(words, n):
    return [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]

# +(n-gramによって重複の有無を調べる)


def mecab_filter2(text, threshold=0.9, min_length=10,  ngram_threshold_2gram=0.20, ngram_threshold_3gram=0.20, ngram_threshold_4gram=0.20):
    if text is None:
        return None

    pos_counter, words, word_counter = parts_count(
        text, return_word_count=True)
    meishi_and_symbol_counts = pos_counter['名詞'] + \
        pos_counter['記号'] + pos_counter['補助記号']
    ratio = meishi_and_symbol_counts / len(words)

    if ratio > threshold and len(text) > min_length:
        return None

    # 2-gramの処理
    ngram_counts_2gram = Counter(n_gram(words, 2))
    total_2grams = sum(ngram_counts_2gram.values())
    most_common_2gram_count = ngram_counts_2gram.most_common(
        1)[0][1] if ngram_counts_2gram else 0
    if total_2grams > 0 and most_common_2gram_count / total_2grams > ngram_threshold_2gram:
        return None

    # 3-gramの処理
    ngram_counts_3gram = Counter(n_gram(words, 3))
    total_3grams = sum(ngram_counts_3gram.values())
    repeated_3grams = sum(
        count for count in ngram_counts_3gram.values() if count > 1)
    if total_3grams > 0 and repeated_3grams / total_3grams > ngram_threshold_3gram:
        return None

    # 4-gramの処理
    ngram_counts_4gram = Counter(n_gram(words, 4))
    total_4grams = sum(ngram_counts_4gram.values())
    repeated_4grams = sum(
        count for count in ngram_counts_4gram.values() if count > 1)
    if total_4grams > 0 and repeated_4grams / total_4grams > ngram_threshold_4gram:
        return None

    return text


### is_remove_num_lines(追加処理)：数字だけの表現すてる

In [13]:
##追加処理
# クリーニングスクリプトに入れ忘れた、日付系を消す処理
import glob
import json
import os
from tqdm import tqdm
import random
# 冒頭の数字だけの行を削除

def is_remove_num_lines(line):
    if len(line) == 0:
        return False
    check_line = line[:20]
    count = sum(c.isdigit() for c in check_line)
    # num_ratio=count/len(check_line)
    num_ratio = count
    # print(num_ratio)
    ratio = 5
    if num_ratio > ratio and check_line.find(":") > 0:
        return False
    if num_ratio > ratio and check_line.find("日") > 0:
        return False
    if num_ratio > ratio and check_line.find("年") > 0:
        return False
    if num_ratio > ratio and check_line.find("-") > 0:
        return False
    if num_ratio > ratio and check_line.find("/") > 0:
        return False
    if num_ratio > ratio and check_line.find("／") > 0:
        return False
    if num_ratio > ratio and check_line.find("月") > 0:
        return False
    return True

100%|██████████| 10000/10000 [00:00<00:00, 91011.16it/s]


### remove_num_line(in line-wise)

In [19]:
##追加処理
# クリーニングスクリプトに入れ忘れた、日付系を消す処理
import glob
import json
import os
from tqdm import tqdm
import random
# 冒頭の数字だけの行を削除

def is_remove_num_lines(line):
    if len(line) == 0:
        return ""
    check_line = line[:20]
    count = sum(c.isdigit() for c in check_line)
    # num_ratio=count/len(check_line)
    num_ratio = count
    # print(num_ratio)
    ratio = 5
    if num_ratio > ratio and check_line.find(":") > 0:
        return ""
    if num_ratio > ratio and check_line.find("日") > 0:
        return ""
    if num_ratio > ratio and check_line.find("年") > 0:
        return ""
    if num_ratio > ratio and check_line.find("-") > 0:
        return ""
    if num_ratio > ratio and check_line.find("/") > 0:
        return ""
    if num_ratio > ratio and check_line.find("／") > 0:
        return ""
    if num_ratio > ratio and check_line.find("月") > 0:
        return ""
    return line

### まとめ line_wise_filter

In [57]:

# lineごとのfilter, clean
def process_line(line):
    """行の清掃とフィルタリングを行う"""
    new_line = noise_line(line)
    new_line = mecab_filter2(new_line)  # n-gramによる重複の除外
    new_line=  is_remove_num_lines(new_line)
    return new_line

def process_paragraph(paragraph):
    """パラグラフ内の各行を処理し、清掃されたパラグラフを返す"""
    new_lines = []
    old_line = ""
    for line in paragraph:
        new_line = process_line(line)
        if new_line and new_line != old_line:
            new_lines.append(new_line)
            old_line = new_line
    return new_lines

def clean_paragraphs(paragraphs):
    """パラグラフのリストを受け取り、重複や不適切な行を除外して再構成する"""
    new_paragraphs = [process_paragraph(paragraph) for paragraph in paragraphs]
    new_paragraphs = ["".join(paragraph) for paragraph in new_paragraphs if paragraph]
    return new_paragraphs

def text_to_cleaned_paragraphs(text):
    """テキストを適切にパラグラフに分割して清掃する"""
    paragraphs = text_to_paragraph_sentences(text)  # パラグラフと文章に分割
    cleaned_paragraphs = clean_paragraphs(paragraphs)  # パラグラフを清掃
    return cleaned_paragraphs

##line_wise---------------------------------------------------------------------
def line_wise(text):
    paragraphs= text_to_paragraph_sentences(text)
    #print(paragraphs)
    if len(paragraphs) !=0:
        text = "\n".join(paragraphs)
    return text


def line_wise(text):
    # パラグラフと文章に分割
    paragraphs = text_to_paragraph_sentences(text)

    new_paragraphs = []
    for paragraph in paragraphs:
        new_lines = []
        old_line = ""
        for line in (paragraph):
            # ルールベースの行チェック
            new_line = rule_base_text_check_clean(line)

            # 名詞だらけのlineを除外
            try:
                # new_line = parts_filter.filter(new_line)
                # new_line = parts_filter.filter2(new_line)  # n-gramによる重複の除外
                new_line = noise_line(line)
                new_line = mecab_filter2(new_line)  # n-gramによる重複の除外
                new_line=  is_remove_num_lines(new_line)
            except:
                pass
            if new_line:
                if new_line == old_line:
                    continue
                old_line = new_line
                # ppl=perp_checker(new_line)
                # print(ppl,new_line)
                new_lines.append(new_line)

        if new_lines:
            new_paragraphs.append(new_lines)

    # 文末が｡などでおわらないパラグラフ中の文章を削除
    # clean_line_endings(new_paragraphs)

    # パラグラフにまとめる
    cleaned_paragraphs = []
    old_lines = ""
    for paragraph in new_paragraphs:
        lines = "".join(paragraph)
        if lines == old_lines:
            continue
        cleaned_paragraphs.append(lines)
        old_lines = lines
    text = "\n".join(cleaned_paragraphs)
    return text

## 関数：rule base　text check_clean

In [56]:
# from .parts_filter import filter

noise_ending_list = """
一覧ページ上部へ
ページ上部へ
上部へ
上へ戻る
"""
noise_ending_list = noise_ending_list.split("\n")
noise_ending_list = [x for x in noise_ending_list if len(x) > 0]


def clean_endings(sent: str):
    for noise_ending in noise_ending_list:
        if sent.endswith(noise_ending):
            return sent[:-len(noise_ending)]
    return sent


noise_header_list = """
トップページ>
"""
noise_header_list = noise_header_list.split("\n")
noise_header_list = [x for x in noise_header_list if len(x) > 0]


def clean_headers(text):
    for noise_header in noise_header_list:
        if text.startswith(noise_header):
            return text[len(noise_header):]
    return text


sentence_endings = ['。', '！', '？', '.', '!', '?', "．", "」", "。"]

# 文頭の見出しを消す


def remove_header(txt, header_list, n_check=30):
    for header in header_list:
        if header in txt[:n_check]:
            for delimiter in header_list:
                txt = txt.split(delimiter)[1:]
                txt = delimiter.join(txt)
                break
    return txt


def rule_base_text_check_clean(text):
    text = remove_header(text, header_list=["|", "】", ">", "]",])
    text = clean_endings(text)
    text = clean_headers(text)
    text = dedup_lines(text)

    # 文章全体で名詞が多い場合は無効と判定
    try:
        if not mecab_filter1(text, threshold=0.7):
            return ""
    except:
        return ""
    for ending in sentence_endings:
        if text.find(ending) > 0:
            return text

    return ""


def dedup_lines(data, check_length=10):

    lines = data.split('\n')
    new_lines = []
    old_line = ""
    for line in lines:
        set_a = set(line[:check_length])
        set_b = set(old_line[:check_length])
        if len(list(set_a-set_b)) < 2:
            continue
        old_line = line
        new_lines.append(line)

    result_text = '\n'.join(new_lines)
    return result_text


## 関数：remove_multi_headers(in line dedup)
使ってない処理がある

In [59]:
begin_symbols = [
    "(",
    "[",
    "{",
    "（",
    "［",
    "｛",
    "〈",
    "《",
    "「",
    "『",
    "【",
    "〔",
    "・",
    "、",
    ",",
    "，",
]

end_symbols = [
    ")",
    "]",
    "}",
    "）",
    "］",
    "｝",
    "〉",
    "》",
    "」",
    "』",
    "】",
    "〕",
]


def is_end_with_begin_symbol(sent):
    for s in begin_symbols:
        if sent.endswith(s):
            return True
    return False


def is_start_with_end_symbol(sent):
    for s in end_symbols:
        if sent.startswith(s):
            return True
    return False


def is_sentence_end(sent):
    end_symbols = [".", "。", "!", "！", "?", "？", "ました", "ます",
                   "れる", "する", "すよ", "です", "」", "』", "さい",]
    for s in end_symbols:
        if sent.endswith(s):
            return True
    return False


def remove_multi_headers(new_texts):
    """
    見出し1
    見出し2
    見出し3
    こんにちは｡

    →
    見出し3
    こんにちは｡

    にする
    """
    new_texts = new_texts.split("\n")
    new_texts = remove_dup_lines(new_texts)
    cleaned_texts = []
    cap_line = ""
    for line in new_texts[::-1]:
        if not is_sentence_end(line):
            # print("line",line)
            if cap_line == "":
                cap_line = line
                # print("cap_line",cap_line)

        else:
            if cap_line != "":
                cleaned_texts.append(cap_line)
                cap_line = ""

            cleaned_texts.append(line)
    cleaned_texts.append(cap_line)
    cleaned_texts = cleaned_texts[::-1]
    cleaned_texts = [i.strip() for i in cleaned_texts]
    cleaned_texts = "\n".join(cleaned_texts)
    return cleaned_texts


def remove_dup_lines(lines, dup_n_threshold=100):
    new_lines = []
    for line in lines:
        # line=line[0].strip()

        # 行の重複を避ける
        if line in new_lines[-dup_n_threshold:]:
            continue
        new_lines.append(line)

    return new_lines


## line end 使ってない

In [60]:

sentence_endings = ['。', '！', '？', '.', '!', '?', "．", "」"]


def clean_line_endings(paragraphs):
    """文末記号以外の文字を削除する"""
    for paragraph in paragraphs:
        if len(paragraph) < 2:
            continue
        for line in paragraph:
            if line[-1] not in sentence_endings:
                paragraph.remove(line)
                # print(line)

# 全て整理


In [55]:
import sys
sys.path.append("/data/fairy_hdd4/takumi/matsuo_llm/Dataset_for_BTM/01web_codes/src/cleaner")
# from .splitter import text_to_paragraph_sentences
# from .text_normalizer import normalize
###from . import text_checker
# from . import rule_based_line_checker
# from . import parts_filter
###from .line_end_cleaner import clean_line_endings
# # from .hojichar_filter import hoji_filter, prob_hoji_filter, prob_filter
# from . import rule_based_text_checker
# from .line_dedup import remove_multi_headers
# # from . import repeated_phrase

# クラシファイアのインポートと初期化
# try:
#     from .TextClassifier import TextClassifier
#     classifier = TextClassifier()
# except:
#     print("error loading TextClassifier. install fasttext to use it")

def clean_text(text, hoji=True):
    """テキスト全体の清掃を行い、整形されたテキストを返す"""
    if hoji:
        text = hoji_filter(text)
        text = prob_hoji_filter(text)
    # paragraphs = text_to_cleaned_paragraphs(text)
    # text = "\n".join(paragraphs)
    #text=normalize(text)
    text=repetition_filter1(text)
    text=repetition_filter2(text)
    text=line_wise(text)
    text = remove_multi_headers(text)
    text = rule_base_text_check_clean(text)
    text = text.strip()
    return text


In [ ]:
from collections import Counter

# 単一文字の繰り返し200----------------------------------------

def repeated_id(text, threshold_ratio=0.3):
    # # 文字の繰り返しをチェックする辞書
    char_count = {}
    # 各文字の繰り返し回数をカウント
    for char in text:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1
    # ratio%以上繰り返される文字があるかチェック
    repeated_chars = [char for char, count in char_count.items(
    ) if count >= threshold_ratio*len(text)]
    # 繰り返される文字がなければmatrix_tempに追加
    if not repeated_chars:
        return text
    return ""


def is_repetitive_japanese(text, thresholds={
    'line_dup': 0.30,
    'paragraph_dup': 0.30,
    'char_in_line_dup': 0.20,
    'char_in_paragraph_dup': 0.20,
    '2-gram': 0.20,
    '3-gram': 0.18,
    '4-gram': 0.16,
    '5-gram': 0.15,
    '6-gram': 0.14,
    '7-gram': 0.13,
    '8-gram': 0.12,
    '9-gram': 0.11,
        '10-gram': 0.10}):
    if text == "":
        return ""
    # 段落と行に分割
    paragraphs = text.split('\n')
    lines = text.replace('\n', ' ').split('。')

    # 段落と行の重複率を計算
    paragraph_dup_rate = calc_dup_rate(paragraphs)
    line_dup_rate = calc_dup_rate(lines)

    # 文字に含まれる重複の割合を計算
    char_in_paragraph_dup_rate = calc_char_dup_rate(paragraphs, text)
    char_in_line_dup_rate = calc_char_dup_rate(lines, text)

    # n-gramの重複率を計算
    ngram_dup_rates = {}
    for n in range(2, 11):
        ngrams = extract_ngrams(text.replace('\n', ''), n)
        if n < 5:
            # 最頻出のn-gramの出現回数を計算
            ngram_dup_rates[n] = calc_max_freq_rate(ngrams)
        else:
            # 2回以上出現するn-gramの総出現回数を計算
            ngram_dup_rates[n] = calc_total_dup_freq_rate(ngrams)

    # 各指標が閾値を超えているかチェック
    if (line_dup_rate > thresholds['line_dup'] or
        paragraph_dup_rate > thresholds['paragraph_dup'] or
        char_in_line_dup_rate > thresholds['char_in_line_dup'] or
        char_in_paragraph_dup_rate > thresholds['char_in_paragraph_dup'] or
            any(ngram_dup_rates[n] > thresholds[f'{n}-gram'] for n in range(2, 11))):
        if line_dup_rate > thresholds['line_dup']:
            print("line_dup")
        if paragraph_dup_rate > thresholds['paragraph_dup']:
            print("paragraph_dup")
        if char_in_line_dup_rate > thresholds['char_in_line_dup']:
            print("char_in_line_dup")
        if char_in_paragraph_dup_rate > thresholds['char_in_paragraph_dup']:
            print('char_in_paragraph_dup')
        for n in range(2, 11):
            if ngram_dup_rates[n] > thresholds[f'{n}-gram']:
                print(f"{n}-gram")
                #print(text)
        return ""
    return text


def calc_dup_rate(items):
    counter = Counter(items)
    total = len(items)
    dup_count = sum(1 for count in counter.values() if count > 1)
    return dup_count / total if total > 0 else 0


def calc_char_dup_rate(items, text):
    counter = Counter(items)
    total_chars = len(text)
    dup_chars = sum(len(item) * (count - 1)
                    for item, count in counter.items() if count > 1)
    return dup_chars / total_chars if total_chars > 0 else 0


def extract_ngrams(text, n):
    return [text[i:i+n] for i in range(len(text)-n+1)]

def calc_max_freq_rate(ngrams):
    counter = Counter(ngrams)
    total = len(ngrams)
    max_freq = max(counter.values())
    return max_freq / total if total > 0 else 0


def calc_total_dup_freq_rate(ngrams):
    counter = Counter(ngrams)
    total = len(ngrams)
    dup_freq = sum(count - 1 for count in counter.values() if count > 1)
    return dup_freq / total if total > 0 else 0

# text = "これはテストです。これはテストです。これはテストです。\nこれもテストです。これもテストです。"

# print(text)
# print(is_repetitive_japanese(text, thresholds))  # 空の文字列を出力
count=0
new_texts=[]
for text in texts:
    #print(text)
    text=extract(text, include_tables=False,target_lang='ja',favour_precision=True)
    if text ==None:
        continue

    #count+=1
    
    # if count<4:
    #     print("extract",text)
    text=is_repetitive_japanese(text)
    if text !="":
        new_texts.append(text)
    else:
        count+=1
        print(f"error{count}")
    # if count>=4:
    #     break
count

In [1]:

def get_n_grams(words: list[str], n: int) -> list[str]:
    return [" ".join(words[i : i + n]) for i in range(len(words) - n + 1)]


def find_duplicates(x: list[str]) -> tuple[int, int]:
    unique_x = set()
    duplicate_chars = 0
    duplicate_elements = 0
    for element in x:
        if element in unique_x:
            duplicate_chars += len(element)
            duplicate_elements += 1

        else:
            unique_x.add(element)
    return duplicate_elements, duplicate_chars

def find_top_duplicate(x: list[str]) -> int:
    counter = Counter()
    for element in x:
        counter[element] += 1
    top_n_gram = counter.most_common(1)[0]
    return len(top_n_gram[0]) * top_n_gram[1]


def find_all_duplicate(words: list[str], n: int) -> int:
    n_words = len(words)
    unique = set()
    repeated_chars, idx = 0, 0
    while idx < n_words - n + 1:
        n_gram = "".join(words[idx : idx + n])
        if n_gram in unique:
            repeated_chars += len(n_gram)
            idx += n
        else:
            unique.add(n_gram)
            idx += 1
    assert repeated_chars <= len("".join(words))
    return repeated_chars


In [ ]:

def filter(self, doc: Document) -> bool | tuple[bool, str]:
    from nltk.tokenize import word_tokenize

    text = doc.text

    paragraphs = self.paragraph_exp.split(text.strip())
    paragraphs_duplicates, char_duplicates = find_duplicates(paragraphs)
    if self.dup_para_frac and paragraphs_duplicates / len(paragraphs) > self.dup_para_frac:
        return False, "dup_para_frac"
    if self.dup_para_char_frac and char_duplicates / len(text) > self.dup_para_char_frac:
        return False, "dup_para_char_frac"

    lines = text.splitlines()
    line_duplicates, char_duplicates = find_duplicates(lines)
    if self.dup_line_frac and line_duplicates / len(lines) > self.dup_line_frac:
        return False, "dup_line_frac"
    if self.dup_line_char_frac and char_duplicates / len(text) > self.dup_line_char_frac:
        return False, "dup_line_char_frac"

    words = word_tokenize(text, language="english")  # TODO we should use language id filter

    for n, n_frac in self.top_n_grams:
        n_grams = get_n_grams(words, n)
        if not n_grams:
            continue
        top_char_length = find_top_duplicate(n_grams)
        if top_char_length / len(text) > n_frac:
            return False, f"top_{n}_gram"

    for n, n_frac in self.dup_n_grams:
        n_duplicates_char = find_all_duplicate(words, n)
        if n_duplicates_char / len(text) > n_frac:
            return False, f"duplicated_{n}_n_grams"

    return True

extract トップページ
製品マニュアル
よくあるご質問[FAQ]
トラブルシューティング
すべてから検索
製品マニュアルから検索
よくある質問[FAQ]から検索
トラブルシューティングから検索
お知らせ
障害情報
マイサイトを編集
サイトの統合・一部ページの移行・分割はできますか
＜＜統合や移行について＞＞各サイトごとにサイトを構築・管理していますので、サイトAとサイトBを統合したり、各サイトの一部のページのみ移行することはできません。 ＜＜分割について＞＞ 分割を行...
よくある質問
全般
ご利用環境・編集関連
コントロールパネル
アプリを起動
各種設定
サイトシアター画面
マイサイトを編集
テンプレートを選ぶ・ブランクサイトから作成
サイトをインポート
サイトエディタ画面
全般
サイト設定
プレビュー・サイトを公開
Dress・ウェブフォント・ページレイアウト
ブロックエディタ画面
ブロックテンプレート
編集
設定
CSS詳細設定
各種パーツ
画像を選択
画像を作成(SiGN Pro)
スライドショー
リンク
アイコン・区切り線
表 テーブル
フォーム
ダウンロード
カスタムタグ
ソーシャル系
ウェブサービス系
ショッピングカートパーツ
その他
SYNC for WebLiFE*5
ソーシャル系/ソーシャルボタン・Instagram
ソーシャル系/タイムライン･Twitterバッジ
ソーシャル系/Page Plugin・コメント欄
ソーシャル系/YouTube・Ustream
ウェブサービス系/Googleマップ・Yahoo!地図
ウェブサービス系/サイト内検索・翻訳
ウェブサービス系/ニュース・ブログ
ウェブサービス系/ドキュメント
サイト・コーナー・ページ設定
サイト設定
コーナー設定
ページ設定
アップロード・FTP設定
サーバー関連
アップロード関連
閲覧
その他・リファレンス
対応言語
その他
各種設定
外部サービス連携/Benchmark Email
サイト管理/かんたんパスワード
サイト管理/独自ドメイン
Copyright(C)BIGLOBE Inc. 1996-2016
filter トップページ
製品マニュアル
よくあるご質問[FAQ]
トラブルシューティング
すべてから検索
製品マニュアルから検索
よくある質問[FAQ]から検索
トラブルシューティングから検索
お知らせ
障害情報
マイサイトを編集
サイトの統合・一部ページの移行・分割はできますか
＜＜統合や移行について＞＞各サイトごとにサイトを構築・管理していますので、サイトAとサイトBを統合したり、各サイトの一部のページのみ移行することはできません。 ＜＜分割について＞＞ 分割を行...
よくある質問
全般
ご利用環境・編集関連
コントロールパネル
アプリを起動
各種設定
サイトシアター画面
マイサイトを編集
テンプレートを選ぶ・ブランクサイトから作成
サイトをインポート
サイトエディタ画面
全般
サイト設定
プレビュー・サイトを公開
Dress・ウェブフォント・ページレイアウト
ブロックエディタ画面
ブロックテンプレート
編集
設定
CSS詳細設定
各種パーツ
画像を選択
画像を作成(SiGN Pro)
スライドショー
リンク
アイコン・区切り線
表 テーブル
フォーム
ダウンロード
カスタムタグ
ソーシャル系
ウェブサービス系
ショッピングカートパーツ
その他
SYNC for WebLiFE*5
ソーシャル系/ソーシャルボタン・Instagram
ソーシャル系/タイムライン･Twitterバッジ
ソーシャル系/Page Plugin・コメント欄
ソーシャル系/YouTube・Ustream
ウェブサービス系/Googleマップ・Yahoo!地図
ウェブサービス系/サイト内検索・翻訳
ウェブサービス系/ニュース・ブログ
ウェブサービス系/ドキュメント
サイト・コーナー・ページ設定
サイト設定
コーナー設定
ページ設定
アップロード・FTP設定
サーバー関連
アップロード関連
閲覧
その他・リファレンス
対応言語
その他
各種設定
外部サービス連携/Benchmark Email
サイト管理/かんたんパスワード
サイト管理/独自ドメイン
Copyright(C)BIGLOBE Inc. 1996-2016
extract 一関市の広告代理店FREE LINE（フリーライン）はお得なクーポン情報をフリーペーパーで紹介しております
広告代理店FREE LINE（フリーライン）では一関市のフリーペーパーを発行しております。お得な情報満載で、皆様のライフスタイルを応援いたします。
一関市のフリーペーパーに情報掲載をお考えなら広告代理店「FREE LINE（フリーライン）」までご連絡ください！
True
filter 
extract Hair Scene023
Creator：Fetish Factory
Format：download
Scene time：00:11:54
Price 999yen
<ホームページ特別価格>
"髪大好き"伸ばせるところまで伸ばしたいという黒髪ロングのゆかりさん。ちょっとサラサラの黒髪ロングを触らせてもらいました。着衣のままで洗髪のお手入れ。
This is an image of woman's hair.
＜当サイト内でのキーワード検索ができます。＞You can search keywords in this site.
＞カテゴリートップへ戻る(Category top)
・ご利用ガイド
ダウンロード販売のシステムはXCREAMに委託しております。
・
ダウンロードについて
・
Engloish FAQ
>
中文(簡体字)
>
中文(繁体字)
>
한국어
・決済手段
お支払はクレジットカードとビットキャッシュ EXがご利用できます。
株式会社ゼウスが提供するクレジットカード決済システムを導入、安心してお買い物いただけます。
Copyright(C) GAGON.COM All right reserved.
filter Hair Scene023
Creator：Fetish Factory
Format：download
Scene time：00:11:54
Price 999yen
<ホームページ特別価格>
"髪大好き"伸ばせるところまで伸ばしたいという黒髪ロングのゆかりさん。ちょっとサラサラの黒髪ロングを触らせてもらいました。着衣のままで洗髪のお手入れ。
This is an image of woman's hair.
＜当サイト内でのキーワード検索ができます。＞You can search keywords in this site.
＞カテゴリートップへ戻る(Category top)
・ご利用ガイド
ダウンロード販売のシステムはXCREAMに委託しております。
・
ダウンロードについて
・
Engloish FAQ
>
中文(簡体字)
>
中文(繁体字)
>
한국어
・決済手段
お支払はクレジットカードとビットキャッシュ EXがご利用できます。
株式会社ゼウスが提供するクレジットカード決済システムを導入、安心してお買い物いただけます。
Copyright(C) GAGON.COM All right reserved.


# 実験
- 特定の処理で実験

In [20]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)


##  少量のWARC形式のものに対しフィルター

In [ ]:
import numpy as np
#from datatrove.utils.text import PUNCTUATION_SET

PUNCTUATION = "!/—”:％１〈&(、━\\【#%「」，】；+^]~“《„';’{|∶´[=-`*．（–？！：$～«〉,><》)?）。…@_.\"}►»" + "".join(
    map(
        chr,
        (x for a, b in ((0, 9), (11, 13), (13, 32), (127, 160)) for x in range(a, b)),
    )
)
PUNCTUATION_SET = set(PUNCTUATION)



STOP_WORDS = ["the", "be", "to", "of", "and", "that", "have", "with"]


class GopherQualityFilter():

    def __init__(
        self,
        min_doc_words: int | None = 50,
        max_doc_words: int | None = 100000,
        min_avg_word_length: int | None = 3,
        max_avg_word_length: int | None = 10,
        max_symbol_word_ratio: float | None = 0.1,
        max_bullet_lines_ratio: float | None = 0.9,
        max_ellipsis_lines_ratio: float | None = 0.3,
        max_non_alpha_words_ratio: float | None = 0.8,
        min_stop_words: int | None = 2,
        stop_words: list[str] | None = None,
        #exclusion_writer: DiskWriter = None,
    ):
        """
        Filter to apply Gopher's quality heuristic rules.
        """
        self.min_doc_words = min_doc_words
        self.max_doc_words = max_doc_words
        self.min_avg_word_length = min_avg_word_length
        self.max_avg_word_length = max_avg_word_length
        self.max_symbol_word_ratio = max_symbol_word_ratio
        self.max_bullet_lines_ratio = max_bullet_lines_ratio
        self.max_ellipsis_lines_ratio = max_ellipsis_lines_ratio
        self.max_non_alpha_words_ratio = max_non_alpha_words_ratio
        self.min_stop_words = min_stop_words
        self.stop_words = set(STOP_WORDS if stop_words is None else stop_words)

    def filter(self, text: str) -> bool | tuple[bool, str]:
        """

        Args:
            doc: Applies the heuristics rules to decide if a document should be REMOVED

        Returns: False if sample.text does not pass any of the the heuristic tests

        """
        from nltk.tokenize import word_tokenize

        words = word_tokenize(text)  # TODO we should use language id filter
        n_words = len(words)
        non_symbol_words = [w for w in words if any(ch not in PUNCTUATION_SET for ch in w)]
        n_non_symbol_words_words = len(non_symbol_words)

        # words < min_doc_words or words > max_doc_words
        #if self.min_doc_words and n_non_symbol_words_words < self.min_doc_words:
         #   print(text)
           # return "",False, "gopher_short_doc"
        if self.max_doc_words and n_non_symbol_words_words > self.max_doc_words:
            return "",False, "gopher_long_doc"

        # mean word length is outside the range of 3 to 10 characters
        avg_n_words = np.mean([len(w) for w in non_symbol_words])
        if self.min_avg_word_length and avg_n_words < self.min_avg_word_length:
            return "",False, "gopher_below_avg_threshold"
        if self.max_avg_word_length and avg_n_words > self.max_avg_word_length:
            return "",False, "gopher_above_avg_threshold"

        # symbol-to-word ratio greater than 0.1 for either the hash symbol or the ellipsis
        if self.max_symbol_word_ratio and text.count("#") / n_words > self.max_symbol_word_ratio:
            return "",False, "gopher_too_many_hashes"
        if self.max_symbol_word_ratio and (text.count("...") + text.count("…")) / n_words > self.max_symbol_word_ratio:
            return "",False, "gopher_too_many_ellipsis"

        # any document with more than 90 % of lines starting with a bullet point,
        # or more than 30 % ending with an ellipsis.
        lines = text.splitlines()
        if (
            self.max_bullet_lines_ratio
            and sum(s.lstrip().startswith("•") or s.lstrip().startswith("-") for s in lines) / len(lines)
            > self.max_bullet_lines_ratio
        ):
            return "",False, "gopher_too_many_bullets"
        if (
            self.max_ellipsis_lines_ratio
            and sum(s.rstrip().endswith("...") or s.rstrip().endswith("…") for s in lines) / len(lines)
            > self.max_ellipsis_lines_ratio
        ):
            return "",False, "gopher_too_many_end_ellipsis"

        # that 80 % of words in a document contain at least one alphabetic character
        if (
            self.max_non_alpha_words_ratio
            and sum([any((c.isalpha() for c in w)) for w in words]) / n_words < self.max_non_alpha_words_ratio
        ):
            return "" ,False, "gopher_below_alpha_threshold"

        # stop word filter
        # if self.min_stop_words and sum(w in self.stop_words for w in words) < self.min_stop_words:
        #     print(text)
        #     return "",False, "gopher_enough_stop_words"

        return text ,True ,"ok"

In [46]:
import nltk
quality_filter=GopherQualityFilter()
new_texts2=[]
for text in new_texts:
    text, _ , m =quality_filter.filter(text)
    print(m)
    if text!="":
        new_texts2.append(text)

ok
gopher_below_alpha_threshold
gopher_above_avg_threshold
gopher_below_alpha_threshold
gopher_above_avg_threshold
ok
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
ok
gopher_above_avg_threshold
ok
gopher_above_avg_threshold
ok
gopher_above_avg_threshold
ok
gopher_above_avg_threshold
gopher_below_alpha_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_below_alpha_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_below_alpha_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
ok
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_below_alpha_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
gopher_above_avg_threshold
ok
ok
gopher_above_avg_threshold
ok
gopher_above_avg_threshold
gopher_above_

In [52]:
new_texts2[5]

'TOPページ\n新規登録\nマイページ\nブログリーダー\n利用規約\n広告掲載\nヘルプ\nNEW\nブロガーのためのSNS「ブログサークル」\nブログに特化した新しいSNSが誕生！ 同じ趣味の仲間とつながったり、自分のブログをみんなにアピールしよう。\n今すぐ登録\nツイート\n投票TOP\n新着トピック\n話題のトピック\n作成・管理\nカテゴリ\nサービスガイド\nＴＯＰ\n乗り物\n飛行機\n受付期間 : 2011年06月11日〜無期限\n自衛隊航空機で好きな機種は？\nF-2\nF-4 ファントム\nF-15 イーグル\nE-2C ホークアイ\nT-4 （ブルーも）\nC-１\nYS-11\nCH-47\nUH-60\nAH-1S コブラ\nUH-1H イロコイ\nP-３C オライオン\nC-130 ハーキュリーズ\nUS-1A\nない\nその他（過去に所属した機種も）\nコメント\n性別：\n【未選択】\n男性\n女性\n年齢：\n【未選択】\n10未満\n10歳代\n20歳代\n30歳代\n40歳代\n50歳代\n60歳代\n70歳代\n80歳代\n90歳代\n100以上\n地域：\n【未選択】\n北海道\n青森\n岩手\n秋田\n宮城\n山形\n福島\n新潟\n栃木\n茨城\n群馬\n埼玉\n東京\n千葉\n神奈川\n山梨\n静岡\n長野\n富山\n岐阜\n愛知\n石川\n福井\n滋賀\n三重\n京都\n奈良\n大阪\n和歌山\n兵庫\n鳥取\n岡山\n島根\n広島\n山口\n香川\n徳島\n高知\n愛媛\n福岡\n佐賀\n大分\n長崎\n熊本\n宮崎\n鹿児島\n沖縄\n【海外】\n©\nさいたかのブログ\nこのトピックをブログパーツにする\nサイズ選択\n自動\n小サイズ\n中サイズ\n大サイズ\n直接リンクを張る場合のURL\nデザインをカスタマイズする\n(細かいデザインの変更が可能です)\n投票TOPへ\n｜\n「飛行機」カテゴリへ\nTOPページ\n新規登録\nマイページ\nブログリーダー\n利用規約\n広告掲載\nヘルプ\nCopyright © 2004 - 2023 "\n@With\n" All rights reserved.'

## WET形式のものに対してフィルタリング

In [62]:
from datasets import load_dataset
from tqdm import tqdm
#mc4の読み込み
dataset = load_dataset('mc4', 'ja',split='train', streaming=True)
n_split=10**2
import os

corpus_dir="corpus/test"
if not os.path.exists(corpus_dir):
    os.makedirs(corpus_dir)

cnt=0
record_id=-1
for record in tqdm(dataset):
    record_id+=1
    
    # if not url_filter(record['url']):
    #     continue
    
    text=record['text']

    #テキストクリーン
    text=clean_text(text)
    if text=="":
        continue

    #記事の判定
    # is_noise=annotator.predict(text)
    # if is_noise==1:
    #     continue

    #ジャンルの判定
    cnt+=1

    d={
        "id":record_id,
        "text":text,
    }
    # file_name=f"{corpus_dir}/{cnt//n_split}.txt"
    # with open(file_name, "a") as f:
    #     f.write(str(d)+"\n")

    if cnt>1000:
        break
    
print(f"圧縮率：{cnt/record_id}")

## フィルターの性能評価

# 他の処理
- Refinedwebを参考にして改良したい。